In [1]:
import random
import re
import pandas as pd
import json
import requests
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

originating_restaurant = None

# Upload JSON Tables

In [2]:
# Create a FileUpload widget that accepts JSON files
uploader = widgets.FileUpload(
    accept='.json',
    multiple=False
)

In [3]:
def handle_uploads(change):
  # Declare global vars because apparently I can't use ones in the broader scope
  global old_sandwiches_table, ingredients_table

  # go through each uploaded file
  for uploaded_file in iter(uploader.value.values()):

      # if the filename matches one of the specified ones
      filename = uploaded_file['metadata']['name']

      # Decode the file content from bytes to a string
      json_str = uploaded_file['content'].decode('utf-8')

      file_json = json.loads(json_str)
      print(f"Successfully uploaded {filename}")

      if filename == "sandwiches.json":
        old_sandwiches_table = file_json
      elif filename == "restaurant_ingredients.json":
        ingredients_table = file_json


In [4]:
# Display the uploader widget
display(uploader)

# Observe changes in the uploader widget
uploader.observe(handle_uploads, names='value')

FileUpload(value={}, accept='.json', description='Upload')

Successfully uploaded restaurant_ingredients.json
Successfully uploaded sandwiches.json


In [6]:
print(old_sandwiches_table)
print(ingredients_table)

[{'_id': 0, 'cuisine': None, 'restaurant': None, 'ingredients': {'breads': {}, 'meats': {'egg': ['egg']}, 'cheeses': {}, 'vegetables': {'onion': ['onion']}, 'condiments': {'oil': ['extra-virgin olive oil'], 'butter': ['butter']}, 'spices': {'black pepper': ['black pepper'], 'salt': ['salt']}}, 'cost': 2.8, 'calories': 343, 'rating': 4, 'dietary_tags': ['vegetarian'], 'review_count': 1, 'reviews': [], 'name': 'Egg Sandwich'}, {'_id': 1, 'cuisine': None, 'restaurant': None, 'ingredients': {'breads': {}, 'meats': {'egg': ['large egg'], 'chicken': ['boneless skinless chicken breast']}, 'cheeses': {}, 'vegetables': {}, 'condiments': {'oil': ['extra-virgin olive oil'], 'butter': ['unsalted butter']}, 'spices': {'black pepper': ['black pepper'], 'salt': ['coarse salt', 'unsalted butter']}}, 'cost': 4.4, 'calories': 503, 'rating': 4, 'dietary_tags': [], 'review_count': 1, 'reviews': [], 'name': 'Egg Sandwich'}, {'_id': 2, 'cuisine': None, 'restaurant': None, 'ingredients': {'breads': {}, 'meat

# Helper Functions

In [39]:
# check if the ingredients are the same between 2 sandwiches
def check_subset(dict1, dict2):
  # if they're equal, regardless of type,
  #   they must be equal and thus each is a subset of the other
  if dict1 == dict2:
    return True

  for subdict1_key, subdict1 in dict1.items():
    # if there's an entry in dict1 not in dict2, it can't be a subset
    if subdict1_key not in dict2.keys():
      return False

    else:
      # if the entry in dict1 is NOT a subset of the respective entry in dict2
      if not check_subset(subdict1, dict2[subdict1_key]):
        return False

  # if all the entries in dict1 were indeed subsets of dict2 entries
  return True

In [40]:
# check if all the ingredients in a sandwich are available at a given restaurant
def available_at_restaurant(sandwich_ingredients, restaurant_ingredients):
  for category_name, category in sandwich_ingredients.items():

    available_ingredients = restaurant_ingredients[category_name]
    for ingredient_name, ingredient in category.items():

      # if the broader ingredient isn't available
      if ingredient_name not in available_ingredients:
        return False

      # if there's an ingredient subtype
      if type(ingredient) == dict:
        for subingredient_name, subingredient in ingredient.items():
          if subingredient_name not in available_ingredients:
            return False

  return True

In [41]:
# estimate the sandwich's associated cost or calorie count at a restaurant
def get_estimate(sandwich_ingredients, restaurant_ingredients, estimate_type="cost"):
  estimate = 0
  for category in sandwich_ingredients:
    for ingredient in sandwich_ingredients[category]:
      if ingredient in restaurant_ingredients[category]:
        try:
          estimate += restaurant_ingredients[category][ingredient][estimate_type]
        except Exception as e:
          print(f"Error getting {estimate_type} estimate: {e}")

  return estimate

# Modify the Tables

Change restaurant name to restaurant list field
- Check sando availability against ingredient lists

Change cost to cost list field
- 1 cost per restaurant
- Default cost estimate if no restaurant

Reviews list field: Points to review objects

## Yes I know this might have been faster with pandas

Type conversion for sandwiches with null values

In [42]:
# make sure the types are correct
for sandwich in old_sandwiches_table:
  sandwich["cost"] = float(sandwich["cost"])
  sandwich["calories"] = int(sandwich["calories"])

  # handle cases of the value being "null"
  if sandwich["cost"] is None:
    sandwich["cost"] = 0
  if sandwich["calories"] is None:
    sandwich["calories"] = 0

Type conversion for ingredients with null (or otherwise bad) values

In [43]:
for restaurant in ingredients_table:
  print(restaurant)
  for category_name, category in restaurant.items():
    print(category_name)
    if category_name != "_id":
      for ingredient_name, ingredient in category.items():
        # print(ingredient_name)
        cost = ingredient["cost"]
        calories = ingredient["calories"]

        # handle cases of the value being "null"
        try:
          cost = float(cost)
        except Exception as e:
          cost = 0

        try:
          calories = int(calories)
        except Exception as e:
          calories = 0

        restaurant[category_name][ingredient_name]["cost"] = cost
        restaurant[category_name][ingredient_name]["calories"] = calories

{'_id': 'default', 'breads': {'baguette': {'cost': 1.5, 'calories': 250}, 'ciabatta': {'cost': 1.8, 'calories': 250}, 'pita': {'cost': 1.0, 'calories': 200}, 'naan': {'cost': 1.2, 'calories': 300}, 'tortilla': {'cost': 0.8, 'calories': 150}, 'roll': {'cost': 0.5, 'calories': 120}, 'sourdough': {'cost': 2.0, 'calories': 220}, 'rye': {'cost': 1.5, 'calories': 200}, 'multigrain': {'cost': 1.5, 'calories': 220}, 'whole wheat': {'cost': 1.0, 'calories': 180}, 'white bread': {'cost': 1.0, 'calories': 160}, 'brioche': {'cost': 2.5, 'calories': 300}, 'focaccia': {'cost': 2.0, 'calories': 250}, 'panini': {'cost': 2.0, 'calories': 200}, 'english muffin': {'cost': 0.8, 'calories': 150}, 'bagel': {'cost': 1.0, 'calories': 250}, 'croissant': {'cost': 1.5, 'calories': 280}, 'milk bread': {'cost': 1.8, 'calories': 220}, 'flatbread': {'cost': 1.0, 'calories': 180}, 'pumpernickel': {'cost': 1.8, 'calories': 190}, 'pretzel roll': {'cost': 1.5, 'calories': 200}, 'potato bread': {'cost': 1.0, 'calories': 

## Build the new list

In [52]:
# create a new list of restructured sandwiches
new_sandwiches_table = []
for sandwich in old_sandwiches_table:
  new_sandwich = dict()
  new_sandwich["_id"] = sandwich["_id"]
  new_sandwich["name"] = sandwich["name"]
  new_sandwich["cuisine"] = sandwich["cuisine"]
  new_sandwich["dietary_tags"] = sandwich["dietary_tags"]
  new_sandwich["ingredients"] = sandwich["ingredients"]
  new_sandwich["restaurants"] = []
  new_sandwich["costs"] = []
  new_sandwich["calories"] = []
  new_sandwich["rating"] = sandwich["rating"]
  new_sandwich["reviews"] = []

  # make sure the ids line up with their list indices
  if len(new_sandwiches_table) != sandwich["_id"]:
    print(f"Error, discontinuity for sandwich {len(new_sandwiches_table)} (sandwich _id = {sandwich['_id']})")
    break

  new_sandwiches_table.append(new_sandwich)

In [53]:
for sandwich in new_sandwiches_table:
  for restaurant in ingredients_table:
    sandwich_ingredients = sandwich["ingredients"]

    # if a sandwich can be made at a given restaurant
    if available_at_restaurant(sandwich_ingredients, restaurant):
      cost_estimate = get_estimate(sandwich_ingredients, restaurant, "cost")

      # account for potential bread or meat mismatches for subway
      if restaurant["_id"] != "default" and cost_estimate < 5:
        cost_estimate += 3.75

      calorie_estimate = get_estimate(sandwich_ingredients, restaurant, "calories")

      # append its associated properties to the new sandwiches table object
      # relies on the assumption that _id == sandwich object's index in the sandwich list
      new_sandwiches_table[sandwich["_id"]]["restaurants"].append(restaurant["_id"])
      new_sandwiches_table[sandwich["_id"]]["costs"].append(cost_estimate)
      new_sandwiches_table[sandwich["_id"]]["calories"].append(calorie_estimate)

  print(f"Restaurants that sell this sandwich: {new_sandwiches_table[sandwich['_id']]['restaurants']}")

  # validate restaurant tagging
  if len(new_sandwiches_table[sandwich["_id"]]["restaurants"]) == 0:
      old_data = old_sandwiches_table[sandwich["_id"]]

      new_sandwiches_table[sandwich["_id"]]["restaurants"].append("default")
      new_sandwiches_table[sandwich["_id"]]["costs"].append(old_data["cost"])
      new_sandwiches_table[sandwich["_id"]]["calories"].append(old_data["calories"])

Restaurants that sell this sandwich: []
Restaurants that sell this sandwich: []
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: ['default']
Restaurants that sell this sandwich: [

In [54]:
def download_json(filename, json_data):
  with open(filename, 'w') as f:
      f.write("[\n")
      for entry in json_data:
        json.dump(entry, f, indent=4)
        f.write(",\n")
      f.write("]")

  # Download the reformatted JSON file
  files.download(filename)

In [56]:
download_json("restaurant_ingredients.json", ingredients_table)
download_json("sandwiches.json", new_sandwiches_table)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>